# Random Forest Model

In [3]:
import numpy as np
import pandas as pd


features = pd.read_csv('./features/features_train.csv')
features.head()


,user_id,product_id,order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,...,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket,up_orders,up_reorder,up_reorder_rate
0,1,17122,1187899,0,4,8,14.0,19.200001,4,8,...,9849.0,0.676907,11,70,6.363636,4.636364,2.110579,1,0,0.000000
1,1,196,1187899,1,4,8,14.0,19.200001,4,8,...,29012.0,0.777843,11,70,6.363636,4.636364,2.110579,11,10,0.909091
2,1,26405,1187899,1,4,8,14.0,19.200001,4,8,...,598.0,0.453374,11,70,6.363636,4.636364,2.110579,3,2,0.666667
3,1,13032,1187899,1,4,8,14.0,19.200001,4,8,...,2581.0,0.661117,11,70,6.363636,4.636364,2.110579,4,3,0.750000
4,1,39657,1187899,1,4,8,14.0,19.200001,4,8,...,4054.0,0.768094,11,70,6.363636,4.636364,2.110579,2,1,0.500000


In [4]:
features.columns

Index([u'user_id', u'product_id', u'order_id', u'in_next_order', u'order_dow',
       u'order_hour_of_day', u'days_since_prior_order',
       u'avg_days_since_prior_order', u'prev_order_dow',
       u'prev_order_hour_of_day', u'prev_days_since_prior_order', u'aisle_id',
       u'department_id', u'p_orders', u'p_reorders', u'p_reorder_rate',
       u'tot_orders', u'tot_prods', u'avg_basket', u'avg_reorder',
       u'std_basket', u'up_orders', u'up_reorder', u'up_reorder_rate'],
      dtype='object')

## Features

* Order features
    * Current Order -  u'order_dow', u'order_hour_of_day', u'days_since_prior_order'
    * Prev Order - u'prev_order_dow',u'prev_order_hour_of_day', u'prev_days_since_prior_order'

* User Order Features
    * u'avg_days_since_prior_order

* Product Features
    * u'aisle_id', u'department_id'
    * u'p_orders', u'p_reorders', u'p_reorder_rate'

* User Features
    * u'tot_orders', u'tot_prods', u'avg_basket', u'avg_reorder', u'std_basket'

* User Product Features
    * u'up_orders', u'up_reorder', u'up_reorder_rate'


    

-----------------------
## 1. Model Building

### Make dummy variables for categorical variables

In [5]:
Y = features.ix[:,3]
X = features.ix[:,4:]

X = pd.get_dummies(X, prefix=['aisle_id','department_id','order_dow',
                              'order_hour_of_day','prev_order_dow','prev_order_hour_of_day']
                   , columns=['aisle_id','department_id','order_dow',
                              'order_hour_of_day','prev_order_dow','prev_order_hour_of_day'])
X.head()

,days_since_prior_order,avg_days_since_prior_order,prev_days_since_prior_order,p_orders,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,...,prev_order_hour_of_day_14,prev_order_hour_of_day_15,prev_order_hour_of_day_16,prev_order_hour_of_day_17,prev_order_hour_of_day_18,prev_order_hour_of_day_19,prev_order_hour_of_day_20,prev_order_hour_of_day_21,prev_order_hour_of_day_22,prev_order_hour_of_day_23
0,14.0,19.200001,14.0,14550.0,9849.0,0.676907,11,70,6.363636,4.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14.0,19.200001,14.0,37298.0,29012.0,0.777843,11,70,6.363636,4.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14.0,19.200001,14.0,1319.0,598.0,0.453374,11,70,6.363636,4.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14.0,19.200001,14.0,3904.0,2581.0,0.661117,11,70,6.363636,4.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.0,19.200001,14.0,5278.0,4054.0,0.768094,11,70,6.363636,4.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convienient scoring function

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix


def print_acc(Y, y_p, model_name):
    print model_name + " {0:.2f}, precision {0:.2f}, recall {0:.2f}, f1-score {0:.2f}"\
                    .format(accuracy_score(Y, y_p)
                            , precision_score(Y, y_p)
                            , recall_score(Y, y_p)
                            , f1_score(Y, y_p))
    print
    print confusion_matrix(Y, y_p, labels =[0,1])

    

### Dummy Classifier - Baseline

In [8]:
from sklearn.dummy import DummyClassifier


dummy = DummyClassifier(strategy='stratified', random_state = 100, constant = None)
dummy.fit(X.values, Y.values)
y_p1 = dummy.predict(X.values)

print_acc(Y.values, y_p1, "DummyClassifier")



Dummy Model , strategy: stratified,accuracy 0.82, precision 0.82, recall 0.82, f1-score 0.82

[[6898652  747185]
 [ 747409   81415]]


### Random Forest Model

## Notes


* 5 Estimators
    RandomForestClassifier accuracy 0.98, precision 0.98, recall 0.98, f1-score 0.98
    f1-score 0.619361997852
* 50 Estimators
    RandomForestClassifier 1.00, precision 1.00, recall 1.00, f1-score 1.00
    f1-score 0.695119773475





In [16]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 50, 
                                oob_score = True, 
                                random_state = 100,
                                verbose = 10, 
                                n_jobs=-1)
forest.fit(X.values, Y.values)

print "Predicting"
y_p2 = forest.predict(X.values)




building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
 building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.2min


building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.6min


building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50


...

building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 27.8min


building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 31.8min finished
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   11.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   22.6s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   25.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   50.7s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:  1.6min finished


TypeError: print_acc() takes exactly 2 arguments (3 given)

In [18]:
print_acc(Y.values, y_p2,"RandomForestClassifier")



RandomForestClassifier 1.00, precision 1.00, recall 1.00, f1-score 1.00

[[7645837       0]
 [   5656  823168]]


### Persist the model

In [20]:
import pickle
pickle.dump(forest, open('./models/RF_nestimatros_50.pkl','wb'))

--------------------------------

## 2. Model Scoring

### F1-Score

In [5]:
import numpy as np
import pandas as pd

def f1_score_dict(predicted, actual):
    """ calculate the f1-socre 
        precision = relevant returned / total returned 
        recall    = relevant returned / total relevant
    """
    precision_scores = []
    recall_scores    = []
    f1_scores        = []

    if len(predicted) != len(actual):
        print "Error: Lenght of prediction and recall doesnt match"
        return 0

    for order_id, products in predicted.items():


        preds = products
        acts  = actual[order_id]
        
        if len(preds) == 0:
            preds = ['None']

        total_returned = len(preds) 
        total_relevant = len(acts)

        relevant_returned = len( set(preds).intersection(set(acts) ) )


        prec = relevant_returned / ( 1.0 * total_returned )
        rec  = relevant_returned / ( 1.0 * total_relevant )

        if rec != 0:
            f1   =  2 * ( (prec * rec) / (prec + rec) )
        else:
            f1   = 0

        precision_scores.append(prec)
        recall_scores.append(rec)
        f1_scores.append( f1)


    return sum(f1_scores) / (1.0 * len(f1_scores))

def get_f1_score(features, y_p):
    
    train = pd.read_csv('./data/order_products__train.csv')

    train.reordered = train.reordered.astype(np.int8)
    train.add_to_cart_order = train.add_to_cart_order.astype(np.int16)

    print "Make a df"
    predicted_df = pd.concat([ features[['user_id','product_id','nxt_order_id'] ] 
                                        , pd.DataFrame(y_p, columns = ['in_basket'])], axis = 1)


    
    actual_ = {}
    print "Actual"
    for row in train.itertuples():
        order_id = str(row.order_id)
        product_id = str(row.product_id)

        if order_id not in actual_:
            actual_[order_id] = []
        actual_[order_id].append(product_id)

    predicted_ = {}

    print "Predicted"
    for row in predicted_df.itertuples():
        order_id = str(row.nxt_order_id)
        product_id = str(row.product_id)


        if order_id not in predicted_:
            predicted_[order_id] = []
        if row.in_basket > 0:
            predicted_[order_id].append(product_id)

    print "find f1-score"

    print f1_score_dict(predicted_, actual_)

In [6]:
def perform_predictions(feature_file, model, find_f1=False, is_train = True,
                        is_prob = True, 
                        prob_threshold = 0.2, make_submission = False, submission_file = 'submission.csv'):
    
    print "Read feature file"
    test_df = pd.read_csv(feature_file)
    
    starting_index 
    X_ = test_df.ix[:,4:]

    print "Make dummy variables"
    X_ = pd.get_dummies(X_, prefix=['aisle_id','department_id','order_dow',
                              'order_hour_of_day','prev_order_dow','prev_order_hour_of_day']
                   , columns=['aisle_id','department_id','order_dow',
                              'order_hour_of_day','prev_order_dow','prev_order_hour_of_day'])
    
    print "Perform Predictions"

    predictions = None
    predictions_prob = None

    if not is_prob:
        predictions = forest_loaded.predict(X_.values)
    else:
        predictions_prob = forest_loaded.predict_proba(X_.values)
        predictions = []
        for rw in predictions_prob:
            if rw[1] >= prob_threshold:
                predictions.append(1)
            else:
                predictions.append(0)
    
    if find_f1:
        print "Get f1 score"
        get_f1_score(test_df, predictions)
    


    if make_submission:

        print "Preparing to make submission"


        final_predictions = pd.concat([ test_df[['user_id','order_id','product_id'] ] 
                                        , pd.DataFrame(predictions, columns = ['in_basket'])], axis = 1)
        
        print "Sort the orders"
        final_predictions.sort(columns = 'order_id', inplace = True)



        final = {}

        for row in final_predictions.itertuples():
            order_id   = row.order_id
            product_id = row.product_id 
            in_basket  = row.in_basket
            if order_id not in final:
                final[order_id] = []
            if in_basket > 0:
                final[order_id].append(str(product_id))

        print "Make submission file"

        i = open(submission_file, 'w')  
        i.write('order_id,products' + '\n')
        for k,v in final.items():
            order_id = str(k)
            product_string = ' '.join(v)
            if len(product_string) < 1:
                product_string = 'None'
            to_write = order_id + ',' + product_string + '\n'
            i.write(to_write)

        i.close()
    




 

### Load the model from disk

In [2]:
import gc

gc.collect()

print "Loading model"
import pickle
forest_loaded = pickle.load(open('./models/RF_nestimatros_50.pkl','r'))


Loading model


In [3]:
forest_loaded

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=True, random_state=100, verbose=10, warm_start=False)

### Model validation on train data

In [7]:

perform_predictions( feature_file = './features/features_train.csv', model = forest_loaded
                    ,find_f1 = True
                    ,is_prob = True
                   , prob_threshold = 0.2)

Read feature file
Make dummy variables


KeyboardInterrupt: 

### Make submission

In [ ]:

perform_predictions(  feature_file = './data/features_test.csv', model = forest_loaded
                    , find_f1 = True
                    , is_prob = True
                    , prob_threshold = 0.2
                    , make_submission = True, submission_file = 'submission_rf.csv')